In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

import sys
sys.path.append("..") 
from tp_utils.data_provider import read_prices, read_data

In [12]:
def log_returns(price):

    return np.diff(np.log(price))

## 0. Load Data

In [13]:
market = 'BNB-BTC'
tf = '1m'

df_raw = read_data(market, tf)
assert df_raw[df_raw.isna()==True].sum().sum() == 0

df = df_raw[-10000:]
rets = log_returns(df['C'])

## ESN

In [89]:
from pyESN import ESN 

n_reservoir= 558
sparsity=0.2
rand_seed=23
spectral_radius = 0.864 #1.2
noise = 0.008034#.0005


esn = ESN(n_inputs = 1,
      n_outputs = 1, 
      n_reservoir = n_reservoir,
      sparsity=sparsity,
      random_state=rand_seed,
      spectral_radius = spectral_radius,
      noise=noise)

In [105]:
res = []
for ii in range(100):
    data = rets
    trainlen = 100
    future = 1
    futureTotal=100
    pred_tot=np.zeros(futureTotal)

    for i in range(0,futureTotal,future):
        pred_training = esn.fit(np.ones(trainlen),data[i:trainlen+i])
        prediction = esn.predict(np.ones(future))
        pred_tot[i:i+future] = prediction[:,0]

    y_true = np.sign(data[trainlen:trainlen+futureTotal])
    y_pred = np.sign(pred_tot)
    res.append(np.mean(y_true == y_pred))

### Grid Searcn

In [117]:
def MSE(yhat, y):
    return np.sqrt(np.mean((yhat.flatten() - y)**2))

def ACC(yhat, y):
    y_true = np.sign(y)
    y_pred = np.sign(yhat)
    return np.mean(y_true == y_pred)
    


In [53]:
%%time
n_reservoir= 500
sparsity   = 0.2
rand_seed  = 23
radius_set = [0.9,  1,  1.1]
noise_set = [ 0.001, 0.004, 0.006]

radius_set = [0.5, 0.7, 0.9,  1,  1.1,1.3,1.5]
noise_set = [ 0.0001, 0.0003,0.0007, 0.001, 0.003, 0.005, 0.007,0.01]



radius_set_size  = len(radius_set)
noise_set_size = len(noise_set)

trainlen = 100
future = 1
futureTotal= 100

loss = np.zeros([radius_set_size, noise_set_size])

for l in range(radius_set_size):
    rho = radius_set[l]
    for j in range(noise_set_size):
        noise = noise_set[j]

        pred_tot=np.zeros(futureTotal)

        esn = ESN(n_inputs = 1,
          n_outputs = 1, 
          n_reservoir = n_reservoir,
          sparsity=sparsity,
          random_state=rand_seed,
          spectral_radius = rho,
          noise=noise)

        for i in range(0,futureTotal,future):
            pred_training = esn.fit(np.ones(trainlen),data[i:trainlen+i])
            prediction = esn.predict(np.ones(future))
            pred_tot[i:i+future] = prediction[:,0]
        
        loss[l, j] = MSE(pred_tot, data[trainlen:trainlen+futureTotal])        
        print('rho = ', radius_set[l], ', noise = ', noise_set[j], ', loss = ', loss[l][j] )

rho =  0.5 , noise =  0.0001 , loss =  0.0008566818127428232
rho =  0.5 , noise =  0.0003 , loss =  0.0008405172615765943
rho =  0.5 , noise =  0.0007 , loss =  0.0008294972531225906
rho =  0.5 , noise =  0.001 , loss =  0.0008264590511716457
rho =  0.5 , noise =  0.003 , loss =  0.0008259017018011381
rho =  0.5 , noise =  0.005 , loss =  0.0008270684426639828
rho =  0.5 , noise =  0.007 , loss =  0.0008269470177020721
rho =  0.5 , noise =  0.01 , loss =  0.0008265653716377131
rho =  0.7 , noise =  0.0001 , loss =  0.0009091999286590972
rho =  0.7 , noise =  0.0003 , loss =  0.0008967287634688495
rho =  0.7 , noise =  0.0007 , loss =  0.0008761812688630412
rho =  0.7 , noise =  0.001 , loss =  0.0008655778637650493
rho =  0.7 , noise =  0.003 , loss =  0.0008409738298064022
rho =  0.7 , noise =  0.005 , loss =  0.0008376896831671447
rho =  0.7 , noise =  0.007 , loss =  0.0008371081070243315
rho =  0.7 , noise =  0.01 , loss =  0.0008376485187025793
rho =  0.9 , noise =  0.0001 , loss 

In [123]:
def f_min(x):
    n_reservoir = int(x[0])
    sparsity = x[1]
    rho   = x[2]
    noise = x[3]
    pred_tot=np.zeros(futureTotal)

    esn = ESN(n_inputs = 1,
      n_outputs = 1, 
      n_reservoir = n_reservoir,
      sparsity=sparsity,
      random_state=rand_seed,
      spectral_radius = rho,
      noise=noise)

    for i in range(0,futureTotal,future):
        pred_training = esn.fit(np.ones(trainlen),data[i:trainlen+i])
        prediction = esn.predict(np.ones(future))
        pred_tot[i:i+future] = prediction[:,0]

    loss = MSE(pred_tot, data[trainlen:trainlen+futureTotal])       
    print('n_reservoir = ', round(n_reservoir), "sparcity =", round(sparsity, 4), 'rho = ', round(rho, 3), ', noise = ', round(noise,6), ', loss = ', loss )
    return loss

## Scipy-opt

#### Test

In [67]:
import numpy as np
from skopt import gp_minimize
np.random.seed(123)
def f(x):
     return (np.sin(5 * x[0] + x[1]) * (1 - np.tanh(x[0] ** 2)) * np.random.randn() * 0.1)

res = gp_minimize(f, [(0.5, 1.5), (0,.01)], n_calls=20)
print("x*=%.2f x*=%.2f f(x*)=%.2f" % (res.x[0], res.x[1], res.fun))


x*=0.75 x*=0.01 f(x*)=-0.05


In [ ]:
%%time
res = gp_minimize(f_min, [(400, 600),(.5, 1.5), (0.00001,.01)], n_calls=100)
print("x*=%.2f,x*=%.2f x*=%.2f f(x*)=%.2f" % (res.x[0], res.x[0], res.x[1], res.fun))

In [70]:
## PSO

In [72]:
from pyswarm import pso

In [124]:
lb = [400.0, 0.1,  0.5,  0.0001]
ub = [600.0, 0.3, 1.5, 0.01]
swarmsize = 60
maxiter = 100
xopt, fopt = pso(f_min, lb, ub, swarmsize=swarmsize, maxiter=maxiter)

n_reservoir =  444 sparcity = 0.2387 rho =  1.045 , noise =  0.001403 , loss =  0.0008483585297090228
n_reservoir =  532 sparcity = 0.2597 rho =  0.712 , noise =  0.007016 , loss =  0.0008188601039957438
n_reservoir =  491 sparcity = 0.2398 rho =  0.61 , noise =  0.002882 , loss =  0.0008512335056085943
n_reservoir =  421 sparcity = 0.119 rho =  0.772 , noise =  0.008153 , loss =  0.0008593517866247433
n_reservoir =  557 sparcity = 0.1185 rho =  1.408 , noise =  0.006383 , loss =  0.0009041209376832733
n_reservoir =  424 sparcity = 0.2921 rho =  0.871 , noise =  0.000296 , loss =  0.0008853781596743906
n_reservoir =  589 sparcity = 0.247 rho =  1.435 , noise =  0.002917 , loss =  0.0008038349398036971
n_reservoir =  496 sparcity = 0.1261 rho =  1.119 , noise =  0.003303 , loss =  0.0008707501509855008
n_reservoir =  548 sparcity = 0.1587 rho =  1.427 , noise =  0.003032 , loss =  0.0008638499141209821
n_reservoir =  590 sparcity = 0.2981 rho =  1.019 , noise =  0.002135 , loss =  0.000

In [88]:
xopt

array([5.58384828e+02, 8.63966429e-01, 8.03437360e-03])